In [10]:
import warnings
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import data_preparation.utils.data_loader as dl
# from graphviz import Source
warnings.filterwarnings('ignore')



X_train, X_test, y_train, y_test = dl.data_loader(
    'data_preparation/db/out/output_std.csv',
)



In [11]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

result = model.fit(X_train, y_train)
y_pred = model.predict(X_test.values)

basic_report = classification_report(y_test, y_pred, output_dict=True)


# cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
# cnf_matrix

# disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix)
# disp.plot()
# plt.show()

y_pred_proba = result.predict(X_test.values)
fpr_basic, tpr_basic, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc_basic = metrics.roc_auc_score(y_test, y_pred_proba)

In [12]:
print(basic_report)

{'0': {'precision': 0.5887888707037643, 'recall': 0.5868678629690048, 'f1-score': 0.5878267973856209, 'support': 2452.0}, '1': {'precision': 0.6013380558835104, 'recall': 0.6032372680615871, 'f1-score': 0.6022861647615294, 'support': 2533.0}, 'accuracy': 0.59518555667001, 'macro avg': {'precision': 0.5950634632936374, 'recall': 0.595052565515296, 'f1-score': 0.5950564810735752, 'support': 4985.0}, 'weighted avg': {'precision': 0.5951654175563815, 'recall': 0.59518555667001, 'f1-score': 0.595173954369207, 'support': 4985.0}}


In [13]:
from sklearn.naive_bayes import ComplementNB

model = ComplementNB()

result = model.fit(X_train, y_train)
y_pred = model.predict(X_test.values)

basic_report = classification_report(y_test, y_pred, output_dict=True)

basic_report

{'0': {'precision': 0.627859477124183,
  'recall': 0.6268352365415987,
  'f1-score': 0.6273469387755102,
  'support': 2452.0},
 '1': {'precision': 0.6393378005518329,
  'recall': 0.6403474141334387,
  'f1-score': 0.6398422090729783,
  'support': 2533.0},
 'accuracy': 0.6337011033099298,
 'macro avg': {'precision': 0.6335986388380079,
  'recall': 0.6335913253375187,
  'f1-score': 0.6335945739242442,
  'support': 4985.0},
 'weighted avg': {'precision': 0.6336918930203188,
  'recall': 0.6337011033099298,
  'f1-score': 0.6336960901623682,
  'support': 4985.0}}

In [14]:
from sklearn.svm import NuSVC

model = NuSVC()

result = model.fit(X_train, y_train)
y_pred = model.predict(X_test)

basic_report = classification_report(y_test, y_pred, output_dict=True)

basic_report

{'0': {'precision': 0.9067245119305857,
  'recall': 0.8523654159869495,
  'f1-score': 0.8787050662182048,
  'support': 2452.0},
 '1': {'precision': 0.8649253731343284,
  'recall': 0.9151204105803395,
  'f1-score': 0.8893151736044504,
  'support': 2533.0},
 'accuracy': 0.8842527582748245,
 'macro avg': {'precision': 0.885824942532457,
  'recall': 0.8837429132836445,
  'f1-score': 0.8840101199113276,
  'support': 4985.0},
 'weighted avg': {'precision': 0.8854853507328084,
  'recall': 0.8842527582748245,
  'f1-score': 0.8840963203825699,
  'support': 4985.0}}

In [15]:
from sklearn.tree import ExtraTreeClassifier

model = ExtraTreeClassifier()

result = model.fit(X_train, y_train)
y_pred = model.predict(X_test)

basic_report = classification_report(y_test, y_pred, output_dict=True)

basic_report

{'0': {'precision': 0.7850660066006601,
  'recall': 0.7761011419249593,
  'f1-score': 0.7805578342904019,
  'support': 2452.0},
 '1': {'precision': 0.7856306130417805,
  'recall': 0.7943150414528227,
  'f1-score': 0.789948959560267,
  'support': 2533.0},
 'accuracy': 0.7853560682046138,
 'macro avg': {'precision': 0.7853483098212203,
  'recall': 0.785208091688891,
  'f1-score': 0.7852533969253345,
  'support': 4985.0},
 'weighted avg': {'precision': 0.7853528968946135,
  'recall': 0.7853560682046138,
  'f1-score': 0.7853296939310375,
  'support': 4985.0}}

In [16]:
from sklearn.tree import DecisionTreeClassifier

# Define model
model = DecisionTreeClassifier(max_depth=10, random_state=75)


param_grid = {
    'max_depth': list(range(5, 20, 1)),
    'max_features': ['auto', 'sqrt', 'log2']
}

search = RandomizedSearchCV(model, param_grid, n_iter=15,
                            cv=10, scoring='accuracy', n_jobs=-1, random_state=1, verbose=2)

# search = RFE(model, n_features_to_select=5, step=1)

# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Fitting 10 folds for each of 15 candidates, totalling 150 fits
Best Score: 0.8097191261703076
Best Hyperparameters: {'max_features': 'sqrt', 'max_depth': 16}


In [ ]:
model = DecisionTreeClassifier(
    max_depth=7, max_features='log2', criterion='entropy')


result = model.fit(X_train, y_train)
y_pred = model.predict(X_test)


print(classification_report(y_test, y_pred))


feat_importances = pd.DataFrame(
    model.feature_importances_, index=X_test.columns, columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
feat_importances.plot(kind='bar', figsize=(8, 6))
plt.show()

In [ ]:
model = DecisionTreeClassifier()


param_grid = {
    'max_depth': list(range(5, 20, 1)),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy', 'log_loss']
}

search = RandomizedSearchCV(model, param_grid, n_iter=25,
                            cv=5, scoring='accuracy', n_jobs=-1, random_state=45, verbose=2)

# search = RFE(model, n_features_to_select=5, step=1)

# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)


# cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
# cnf_matrix

# disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix)
# disp.plot()
# plt.show()

best_random = result.best_estimator_
y_pred_test = best_random.predict(X_test)
print(classification_report(y_test, y_pred_test))

fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_test)
auc = metrics.roc_auc_score(y_test, y_pred_test)
# plt.plot(fpr, tpr, label="data 1, auc="+str(auc))
# plt.plot(fpr_basic, tpr_basic, label="Basic, auc=" +
#          str(auc_basic), color='blue')
# plt.legend(loc=4)
# plt.show()

In [ ]:
# Decision tree with RFECV
from sklearn.tree import export_graphviz
from graphviz import Source

model = DecisionTreeClassifier(
    max_depth=7, max_features='log2', criterion='entropy', random_state=45)

cv = StratifiedKFold(3)

rfecv = RFECV(model, cv=cv, scoring='accuracy', step=1)

rfecv.fit(X_train, y_train)

y_pred = rfecv.predict(X_test)

print('Optimal number of features : %d' % rfecv.n_features_)

ranks = pd.DataFrame(
    rfecv.ranking_, index=X.columns, columns=['Rank'])

print(ranks.sort_values(by='Rank', ascending=True))

selected_features = ranks[ranks['Rank'] == 1].index.values.tolist()


# print(rfecv.feature_names_in_)

ranks = pd.DataFrame(
    rfecv.ranking_, index=X.columns, columns=['Rank'])

ranks_1 = ranks[ranks['Rank'] == 1]

# selected_features = ranks[ranks['Rank'] == 1].index.values.tolist()

export_graphviz(rfecv.estimator_, out_file='tree.dot', feature_names=ranks_1.index.values,
                class_names=['0', '1'], rounded=True, filled=True)


print(selected_features)


# print(ranks_1)

# print(ranks.sort_values(by='Rank', ascending=True))

# selected_features = ranks_1

# rfecv.support_rfecv_df = pd.DataFrame(rfecv.ranking_, index=X.columns, columns=[
#                                       'Rank']).sort_values(by='Rank', ascending=True)
# plt.style.use('ggplot')
# plt.figure(figsize=(12, 6))
# plt.xlabel('Number of features selected')
# plt.ylabel('Cross validation score (nb of correct classifications)')
# plt.plot(range(1, len(rfecv.cv_results_[
#          "mean_test_score"]) + 1), rfecv.cv_results_["mean_test_score"])
# plt.title("Recursive Feature Elimination \nwith correlated features")
# plt.show()


print(classification_report(y_test, y_pred))


# cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
# cnf_matrix


# disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix)
# disp.plot()
# plt.grid(False)
# plt.show()

y_pred_proba = result.predict_proba(X_test)[::, 1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.grid(True)
plt.plot(fpr, tpr, label="Optimized, auc="+str(auc))
plt.plot(fpr_basic, tpr_basic, label="Basic, auc=" +
         str(auc_basic), color='blue')
plt.legend(loc=4)
plt.show()

In [ ]:
# X = df[selected_features]
# X_train, X_test, y_train, y_test = train_test_split(
#     X, Y, test_size=0.1, random_state=45, stratify=Y)

In [ ]:
# model = DecisionTreeClassifier(
#     max_depth=7, max_features='log2', criterion='entropy')


# result = model.fit(X_train, y_train)
# y_pred = model.predict(X_test)


# print(classification_report(y_test, y_pred))


# plt.grid(True)
# plt.plot(fpr, tpr, label="data 1, auc="+str(auc))
# plt.plot(fpr_basic, tpr_basic, label="Basic, auc=" +
#          str(auc_basic), color='blue')
# plt.legend(loc=4)
# plt.show()

# export_graphviz(model, out_file='tree.dot', feature_names=X.columns,
#                 class_names=['0', '1'], rounded=True, filled=True)


# feat_importances = pd.DataFrame(
#     model.feature_importances_, index=X_test.columns, columns=["Importance"])
# feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
# feat_importances.plot(kind='bar', figsize=(8, 6))
# plt.show()


# y_pred_proba = result.predict(X_test)
# fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
# auc = metrics.roc_auc_score(y_test, y_pred_proba)

# plt.grid(True)
# plt.plot(fpr, tpr, label="data 1, auc="+str(auc))
# plt.plot(fpr_basic, tpr_basic, label="Basic, auc=" +
#          str(auc_basic), color='blue')
# plt.legend(loc=4)
# plt.show()

In [ ]:
import json

filename = 'results.json'

final_results = {
    "report_basic": basic_report,
    # "fpr_basic": fpr_basic.tolist(),
    # "tpr_basic": tpr_basic.tolist(),
    "auc_basic": float(auc_basic),
    "report_optimized": classification_report(y_test, y_pred, output_dict=True),
    # "fpr_optimized": fpr.tolist(),
    # "tpr_optimized": tpr.tolist(),
    "auc_optimied": float(auc),
    "selected_features": selected_features,
}

try:
    with open(filename, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    data = {}

data['DecisionTree'] = final_results

try:
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)
except json.JSONDecodeError as e:
    print(f"Error: {e}")


data['DecisionTree'] = (final_results)


with open('results.json', 'w') as file:
    json.dump(data, file, indent=4)